### RDD

Let's create our first [RDD](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD). A Resilient Distributed Dataset (RDD), the basic abstraction in Spark. Represents an immutable, partitioned collection of elements that can be operated on in parallel. Spark applications run as independent sets of processes on a cluster, coordinated by the SparkContext object in your main program (called the driver).

In [1]:
dataset = sc.parallelize(range(1000))

A RDD is divided in partitions.

In [2]:
dataset.toDebugString()

'(16) ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:392 []'

[RDD](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD) support two type of operations: transformations and actions. 

###RDD Transformations

A transformation creates a new dataset from an existing one.

In [3]:
squared = dataset.map(lambda x: x*x)

The map operation is a method of the RDD, while the lambda function passed as argument is a common Python function. As long the code is serializable there are no restrictions on the kind of Python code that can be executed. Note that all transformations in Spark are lazy; an action is required to actually realize a transformation, which explains why the map returned so quickly.

Let's keep all multiples of 3.

In [4]:
dataset.filter(lambda x: x % 3 == 0)

PythonRDD[1] at RDD at PythonRDD.scala:43

Let's get a 10% sample without replacement.

In [5]:
dataset.sample(False, 0.1)

PythonRDD[2] at RDD at PythonRDD.scala:43

### RDD Actions

An action returns a value after running a computation on the dataset.

In [6]:
squared.count()

1000

Get the first element.

In [7]:
squared.first()

0

Get the first k elements.

In [8]:
squared.take(5)

[0, 1, 4, 9, 16]

Get a Python list of all elements.

In [9]:
squared.collect()[:5]

[0, 1, 4, 9, 16]

Note that you can't access an arbitrary row of the RDD.

Reduce the elements of the RDD.

In [10]:
squared.reduce(lambda x, y: x + y)

332833500

###RDD Caching

Rerunning an action will retrigger the computation.

In [11]:
squared.count()

1000

We can cache a RDD so that successive accesses to it don't recompute the whole thing.

In [12]:
squared.cache()

PythonRDD[4] at RDD at PythonRDD.scala:43

In [13]:
squared.count()

1000

In [14]:
squared.count()

1000

Once we are done with a dataset we can remove it from memory.

In [15]:
squared.unpersist()

PythonRDD[4] at RDD at PythonRDD.scala:43

Doesn't seem like much right now but once you start handling some real datasets you will appreciate the speed boost, assuming it fits in memory that is. Check out this [documentation](http://spark.apache.org/docs/latest/programming-guide.html#rdd-persistence) to learn more about the persistence policies.

###Key-value pairs

Most Spark operations work on RDDs of any types but few are reserved for key-value pairs, the most common ones are distributed “shuffle” operations, such as grouping or aggregating the elements by a key.

In [16]:
grouped = dataset.map(lambda x: (x % 2 == 0, x))

We can reduce by key,

In [17]:
grouped.reduceByKey(lambda x, y: x + y).collectAsMap()

{False: 250000, True: 249500}

... group by key,

In [18]:
grouped.groupByKey().collectAsMap()

{False: <pyspark.resultiterable.ResultIterable at 0x7fec428b4910>,
 True: <pyspark.resultiterable.ResultIterable at 0x7fec428b4c50>}

or count by key (action).

In [19]:
grouped.countByKey()

defaultdict(<type 'int'>, {False: 500, True: 500})

Check out the [documentation](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions) for a complete list of operations on key-value pairs.

### Scalars

Some simple stats operations are reserved for RDDs of scalars.

In [20]:
dataset.mean()

499.5

In [21]:
dataset.sum()

499500

In [22]:
dataset.stdev()

288.67499025720952

You can also create a simple textual histogram.

In [23]:
dataset.histogram(10)

([0.0,
  99.9,
  199.8,
  299.70000000000005,
  399.6,
  499.5,
  599.4000000000001,
  699.3000000000001,
  799.2,
  899.1,
  999],
 [100, 100, 100, 100, 100, 100, 100, 100, 100, 100])

Check out the [documentation](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.DoubleRDDFunctions) for a complete list of operations on doubles.

##### Exercises

1) Write some code that prints the numbers from 1 to 1000. But for multiples of three print "Fizz" instead of the number and for the multiples of five print "Buzz". For numbers which are multiples of both three and five print "FizzBuzz".